In [2]:
import pandas as pd
import joblib # Untuk load file .pkl
from sentence_transformers import SentenceTransformer, util
from rank_bm25 import BM25Okapi
import string
import nltk
from nltk.corpus import stopwords
import torch
import os

# --- KONFIGURASI PATH ---
if 'ROOT_DIR' not in locals():
    ROOT_DIR = os.getcwd()
    if not os.path.exists(os.path.join(ROOT_DIR, 'models')):
        ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

MODEL_DIR = os.path.join(ROOT_DIR, 'models')
DATA_DIR = os.path.join(ROOT_DIR, 'data', 'processed')

# FILE-FILE YANG DIBUTUHKAN
CSV_PATH = os.path.join(DATA_DIR, 'dataset_training_FULL_COMPLETE.csv')
EMB_FILE = os.path.join(MODEL_DIR, 'corpus_embeddings.pt')
RF_PATH = os.path.join(MODEL_DIR, 'randomforest (2).pkl') # Model Teman Anda

# URUTAN FITUR STANDAR UNTUK RANDOM FOREST (asumsi urutan standar)
RF_FEATURES = ['sbert_sim', 'bm25_score', 'overlap_score', 'jaccard_score'] 

print("✅ Path berhasil diset.")

✅ Path berhasil diset.


In [3]:
print("⚙️ Memuat Dataset Lengkap...")

if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"File {CSV_PATH} tidak ditemukan!")

# 1. Baca CSV
df_full = pd.read_csv(CSV_PATH)
df_full.columns = df_full.columns.str.strip().str.lower()

# 2. Ambil Tafsir Unik (Corpus Pencarian)
df_index = df_full.drop_duplicates(subset=['text']).copy()
unique_tafsirs = df_index['text'].astype(str).tolist()

# 3. Buat Kamus Metadata
metadata_map = {}
for _, row in df_index.iterrows():
    key = str(row['text']).strip()
    metadata_map[key] = {
        'lokasi': row.get('ayat_asal', '?'),
        'arabic': row.get('arabic', ''),
        'trans': row.get('translation', '')
    }

print(f"✅ Data Termuat: {len(unique_tafsirs)} dokumen tafsir unik.")

⚙️ Memuat Dataset Lengkap...
✅ Data Termuat: 6031 dokumen tafsir unik.


In [4]:
print("⚙️ Memuat Embedding & SBERT...")

# 1. Load Embedding Lama (.pt)
if os.path.exists(EMB_FILE):
    corpus_embeddings = torch.load(EMB_FILE, map_location='cpu')
    print(f"✅ Embedding Lama Berhasil Diload.")
else:
    raise FileNotFoundError("File corpus_embeddings.pt hilang!")

# 2. Load SBERT (Hanya untuk encode query)
SBERT_FOLDER = os.path.join(MODEL_DIR, 'sbert_finetuned_quran')
try:
    sbert_model = SentenceTransformer(SBERT_FOLDER, device='cpu')
    print("✅ SBERT Loaded.")
except:
    sbert_model = SentenceTransformer(SBERT_FOLDER, device='cpu')

⚙️ Memuat Embedding & SBERT...
✅ Embedding Lama Berhasil Diload.


The tokenizer you are loading from 'c:\Kuliah ITS Farhan\Semester 3\A_Final_Project\ML_DM\fp-quran-ir-query-tafsir\models\sbert_finetuned_quran' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


✅ SBERT Loaded.


In [5]:
print("⚙️ Memuat Model Random Forest...")

if not os.path.exists(RF_PATH):
    raise FileNotFoundError(f"Model {RF_PATH} tidak ditemukan.")

try:
    # Coba load pakai joblib (paling umum untuk model sklearn)
    rf_model = joblib.load(RF_PATH)
    print("✅ Model RF berhasil dimuat via Joblib.")
except Exception as e:
    # Fallback/Error handling jika gagal
    print(f"❌ Gagal memuat model RF: {e}")
    raise SystemExit("Model RF gagal dimuat. Cek instalasi joblib/sklearn Anda.")

⚙️ Memuat Model Random Forest...
❌ Gagal memuat model RF: node array from the pickle has an incompatible dtype:
- expected: {'names': ['left_child', 'right_child', 'feature', 'threshold', 'impurity', 'n_node_samples', 'weighted_n_node_samples', 'missing_go_to_left'], 'formats': ['<i8', '<i8', '<i8', '<f8', '<f8', '<i8', '<f8', 'u1'], 'offsets': [0, 8, 16, 24, 32, 40, 48, 56], 'itemsize': 64}
- got     : [('left_child', '<i8'), ('right_child', '<i8'), ('feature', '<i8'), ('threshold', '<f8'), ('impurity', '<f8'), ('n_node_samples', '<i8'), ('weighted_n_node_samples', '<f8')]


c:\Users\Farhan\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.2.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


SystemExit: Model RF gagal dimuat. Cek instalasi joblib/sklearn Anda.

C:\Users\Farhan\AppData\Roaming\Python\Python314\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report, 
    roc_auc_score, 
    precision_recall_curve,
    confusion_matrix
)

# --- 1. SETUP & LOAD DATA ---
if 'ROOT_DIR' not in locals(): ROOT_DIR = os.getcwd()
MODEL_DIR = os.path.join(ROOT_DIR, 'models')
DATA_PATH = os.path.join(ROOT_DIR, 'data', 'processed', 'dataset_training_FULL_COMPLETE.csv')

print("🚀 MEMULAI TRAINING RANDOM FOREST...")

# Load Data Matang
df = pd.read_csv(DATA_PATH, on_bad_lines='skip')
df.columns = df.columns.str.strip().str.lower().str.replace('"', '')

# Definisi Fitur (Diambil langsung dari CSV)
FEATURES = ['sbert_sim', 'bm25_score', 'overlap_score', 'jaccard_score']
TARGET = 'label'
TARGET_RECALL = 0.85 # Target kita adalah 85% kelengkapan (Recall)

# Preprocessing: Pastikan tipe data benar dan tidak ada NaN
for col in FEATURES + [TARGET]:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.dropna(subset=FEATURES + [TARGET])

print(f"   -> Data Siap: {len(df)} baris.")
print(f"   -> Fitur Digunakan: {FEATURES}")

# --- 2. SPLIT DATA ---
X = df[FEATURES]
y = df[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- 3. TRAINING RANDOM FOREST ---
print("⚙️ Melatih Model Random Forest...")

model_rf = RandomForestClassifier(
    n_estimators=300, # Jumlah pohon yang cukup besar
    max_depth=10,     # Batasan kedalaman
    min_samples_leaf=5, # Mencegah overfitting
    class_weight='balanced', # KUNCI UTAMA: Menangani ketidakseimbangan kelas
    random_state=42,
    n_jobs=-1 # Gunakan semua core CPU
)

model_rf.fit(X_train, y_train)
print("   -> Training Selesai.")

# --- 4. EVALUASI & MENCARI THRESHOLD OPTIMAL ---
print("📊 Evaluasi Model & Mencari Threshold Optimal...")

y_prob = model_rf.predict_proba(X_test)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_test, y_prob)

# Cari Threshold Terbaik untuk Target Recall
valid_indices = np.where(recall >= TARGET_RECALL)[0]

if len(valid_indices) > 0:
    idx = valid_indices[-1]
    best_thresh = thresholds[idx]
    rec_at_thresh = recall[idx]
    prec_at_thresh = precision[idx]
else:
    best_thresh = 0.5
    rec_at_thresh = 0
    prec_at_thresh = 0

print(f"   -> Threshold Optimal (Target Recall {TARGET_RECALL*100}%): {best_thresh:.4f}")
print(f"   -> Recall Dicapai: {rec_at_thresh:.4f}")
print(f"   -> Precision: {prec_at_thresh:.4f}")
print(f"   -> ROC-AUC Score: {roc_auc_score(y_test, y_prob):.4f}")

# --- 5. SIMPAN MODEL & THRESHOLD ---
model_file = os.path.join(MODEL_DIR, 'randomforest_custom.pkl')
thresh_file = os.path.join(MODEL_DIR, 'threshold_rf.txt')

# Simpan model Random Forest (Joblib lebih disarankan untuk sklearn)
joblib.dump(model_rf, model_file)
with open(thresh_file, 'w') as f:
    f.write(str(best_thresh))

print(f"✅ MODEL DISIMPAN: {model_file}")

🚀 MEMULAI TRAINING RANDOM FOREST...
   -> Data Siap: 170372 baris.
   -> Fitur Digunakan: ['sbert_sim', 'bm25_score', 'overlap_score', 'jaccard_score']
⚙️ Melatih Model Random Forest...
   -> Training Selesai.
📊 Evaluasi Model & Mencari Threshold Optimal...
   -> Threshold Optimal (Target Recall 85.0%): 0.2943
   -> Recall Dicapai: 0.8501
   -> Precision: 0.3682
   -> ROC-AUC Score: 0.8067
✅ MODEL DISIMPAN: c:\Kuliah ITS Farhan\Semester 3\A_Final_Project\ML_DM\fp-quran-ir-query-tafsir\models\randomforest_custom.pkl


In [7]:
import pandas as pd
import joblib 
from sentence_transformers import SentenceTransformer, util
from rank_bm25 import BM25Okapi
import string
import nltk
from nltk.corpus import stopwords
import torch
import os

# --- KONFIGURASI PATH ---
if 'ROOT_DIR' not in locals():
    ROOT_DIR = os.getcwd()
    if not os.path.exists(os.path.join(ROOT_DIR, 'models')):
        ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

MODEL_DIR = os.path.join(ROOT_DIR, 'models')
DATA_DIR = os.path.join(ROOT_DIR, 'data', 'processed')

# FILE-FILE YANG DIBUTUHKAN
CSV_PATH = os.path.join(DATA_DIR, 'dataset_training_FULL_COMPLETE.csv')
EMB_FILE = os.path.join(MODEL_DIR, 'corpus_embeddings.pt')
RF_PATH = os.path.join(MODEL_DIR, 'randomforest_custom.pkl') # MODEL CUSTOM ANDA
RF_THRESHOLD_FILE = os.path.join(MODEL_DIR, 'threshold_rf.txt')

# URUTAN FITUR WAJIB (Sesuai saat training)
RF_FEATURES = ['sbert_sim', 'bm25_score', 'overlap_score', 'jaccard_score'] 

print("✅ Path berhasil diset. Siap menguji model custom.")

✅ Path berhasil diset. Siap menguji model custom.


In [8]:
print("⚙️ Memuat Dataset Lengkap...")

if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"File {CSV_PATH} tidak ditemukan!")

# 1. Baca CSV
df_full = pd.read_csv(CSV_PATH)
df_full.columns = df_full.columns.str.strip().str.lower()

# 2. Ambil Tafsir Unik (Corpus Pencarian)
df_index = df_full.drop_duplicates(subset=['text']).copy()
unique_tafsirs = df_index['text'].astype(str).tolist()

# 3. Buat Kamus Metadata
metadata_map = {}
for _, row in df_index.iterrows():
    key = str(row['text']).strip()
    metadata_map[key] = {
        'lokasi': row.get('ayat_asal', '?'),
        'arabic': row.get('arabic', ''),
        'trans': row.get('translation', '')
    }

print(f"✅ Data Termuat: {len(unique_tafsirs)} dokumen tafsir unik.")

⚙️ Memuat Dataset Lengkap...
✅ Data Termuat: 6031 dokumen tafsir unik.


In [9]:
print("⚙️ Memuat Embedding & SBERT...")

# 1. Load Embedding Lama (.pt)
if os.path.exists(EMB_FILE):
    corpus_embeddings = torch.load(EMB_FILE, map_location='cpu')
    print(f"✅ Embedding Lama Berhasil Diload.")
else:
    raise FileNotFoundError("File corpus_embeddings.pt hilang!")

# 2. Load SBERT (Hanya untuk encode query)
SBERT_FOLDER = os.path.join(MODEL_DIR, 'sbert_finetuned_quran')
try:
    sbert_model = SentenceTransformer(SBERT_FOLDER, device='cpu')
    print("✅ SBERT Loaded.")
except:
    sbert_model = SentenceTransformer(SBERT_FOLDER, device='cpu')

⚙️ Memuat Embedding & SBERT...
✅ Embedding Lama Berhasil Diload.


The tokenizer you are loading from 'c:\Kuliah ITS Farhan\Semester 3\A_Final_Project\ML_DM\fp-quran-ir-query-tafsir\models\sbert_finetuned_quran' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


✅ SBERT Loaded.


In [10]:
print("⚙️ Memuat Model Random Forest Custom...")

if not os.path.exists(RF_PATH):
    raise FileNotFoundError(f"Model {RF_PATH} tidak ditemukan.")
    
# 1. Load Model
try:
    rf_model = joblib.load(RF_PATH)
    print("✅ Model RF berhasil dimuat via Joblib.")
except Exception as e:
    print(f"❌ Gagal memuat model RF: {e}")
    raise SystemExit("Model RF gagal dimuat.")
    
# 2. Load Threshold Optimal
if os.path.exists(RF_THRESHOLD_FILE):
    with open(RF_THRESHOLD_FILE, 'r') as f:
        RF_THRESHOLD = float(f.read().strip())
    print(f"✅ Threshold Optimal Diload: {RF_THRESHOLD:.4f}")
else:
    RF_THRESHOLD = 0.5
    print("⚠️ Threshold file hilang. Menggunakan default 0.5.")

⚙️ Memuat Model Random Forest Custom...
✅ Model RF berhasil dimuat via Joblib.
✅ Threshold Optimal Diload: 0.2943


In [11]:
print("⚙️ Menyiapkan BM25...")

try:
    nltk.download('stopwords', quiet=True)
    stop_words = set(stopwords.words('indonesian'))
except:
    stop_words = set(['yang', 'dan', 'di'])

def clean_tokens(text):
    text = str(text).lower().translate(str.maketrans('', '', string.punctuation))
    return [w for w in text.split() if w not in stop_words]

# Build Index BM25
corpus_tokens = [clean_tokens(t) for t in unique_tafsirs]
bm25 = BM25Okapi(corpus_tokens)

# Pastikan fungsi clean_tokens dapat diakses di cell lain
clean_func = clean_tokens 

print("✅ BM25 Siap.")

⚙️ Menyiapkan BM25...
✅ BM25 Siap.


In [12]:
def cari_random_forest(query_text, threshold=RF_THRESHOLD):
    print(f"\n🔍 QUERY: '{query_text}'")
    print(f"⚙️ Threshold yang digunakan: {threshold:.4f}")
    print("=" * 60)
    
    # 1. Retrieval (SBERT)
    query_vec = sbert_model.encode(query_text, convert_to_tensor=True)
    hits = util.semantic_search(query_vec, corpus_embeddings, top_k=50)[0]
    
    candidates = []
    q_toks = clean_func(query_text)
    
    for hit in hits:
        idx = hit['corpus_id']
        txt = unique_tafsirs[idx]
        
        # Hitung Fitur
        t_toks = clean_func(txt)
        sq, st = set(q_toks), set(t_toks)
        ov = len(sq & st) / len(sq) if sq else 0
        jac = len(sq & st) / (len(sq | st) + 1e-9)
        bm25_s = bm25.get_batch_scores(q_toks, [idx])[0]
        
        candidates.append({
            'text_key': txt,
            'sbert_sim': hit['score'],
            'bm25_score': bm25_s,
            'overlap_score': ov,
            'jaccard_score': jac
        })
    
    # 2. Prediksi dengan Random Forest
    df_cand = pd.DataFrame(candidates)
    
    # Susun kolom sesuai urutan training
    X_pred = df_cand[RF_FEATURES]
    
    # Random Forest predict_proba (ambil probabilitas kelas 1)
    try:
        scores = rf_model.predict_proba(X_pred)[:, 1]
    except AttributeError:
        # Jika model dilatih sebagai regresi atau format beda
        scores = rf_model.predict(X_pred)
    
    df_cand['final_score'] = scores
    
    # 3. Filter & Tampilkan
    results = df_cand[df_cand['final_score'] > threshold].sort_values('final_score', ascending=False)
    
    if len(results) == 0:
        print("❌ Tidak ada hasil relevan (Coba turunkan threshold).")
    else:
        print(f"✅ Top 3 Hasil (Engine: Random Forest Custom):\n")
        
        for i, row in enumerate(results.head(3).iterrows()):
            _, data = row
            key = str(data['text_key']).strip()
            score = data['final_score']
            info = metadata_map.get(key)
            
            print(f"🏅 [RANK {i+1}] Skor: {score:.4f}")
            if info:
                print(f"📍 {info['lokasi']}")
                print(f"🇸🇦 {info['arabic'][:60]}...")
                print(f"🇮🇩 {info['trans'][:100]}...")
            print(f"💬 TAFSIR: {key[:100]}...")
            print("-" * 50)

In [ ]:
cari_random_forest("Kisah sapi betina bani israil")
cari_random_forest("Hukum warisan bagi perempuan")


🔍 QUERY: 'Kisah sapi betina bani israil'
⚙️ Threshold yang digunakan: 0.2943
✅ Top 3 Hasil (Engine: Random Forest Custom):

🏅 [RANK 1] Skor: 0.8939
📍 QS. Ṭāhā : Ayat 96
🇸🇦 قَالَ بَصُرْتُ بِمَا لَمْ يَبْصُرُوْا بِهٖ فَقَبَضْتُ قَبْضَ...
🇮🇩 Dia (Samiri) menjawab, “Aku melihat sesuatu yang tidak mereka lihat. Kemudian, aku ambil segenggam (...
💬 TAFSIR: Mendapat pertanyaan dari Nabi Musa, dia menjawab, “Aku melihat dan mengetahui sesuatu yang tidak mer...
--------------------------------------------------
🏅 [RANK 2] Skor: 0.8514
📍 QS. Al-Baqarah  : Ayat 51
🇸🇦 وَاِذْ وٰعَدْنَا مُوْسٰىٓ اَرْبَعِيْنَ لَيْلَةً ثُمَّ اتَّخَ...
🇮🇩 (Ingatlah) ketika Kami menjanjikan (petunjuk Taurat) kepada Musa (melalui munajat selama) empat pulu...
💬 TAFSIR: Setelah menerima nikmat dalam bentuk penyelamatan dari dua bencana pembunuhan dan tenggelam di Laut ...
--------------------------------------------------
🏅 [RANK 3] Skor: 0.8396
📍 QS. Al-Mā'idah : Ayat 72
🇸🇦 لَقَدْ كَفَرَ الَّذِيْنَ قَالُوْٓا اِنَّ اللّٰ